In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df = pd.read_csv("../Data/final_df2.csv")

In [8]:
df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)

In [9]:
df.head()

,name,appid,developer_x,publisher_x,positive,negative,userscore,owners,average_forever,average_2weeks,...,pos/neg rate,url,all_reviews,release_date,developer_y,publisher_y,popular_tags,genre,original_price,images
0,Dota 2,570.0,valve,Valve,1173499.0,214974.0,0.0,"100,000,000 .. 200,000,000",31355.0,1943.0,...,0.845172,https://store.steampowered.com/app/570/dota_2/,8.0,2013,valve,"valve,valve",free to play moba multiplayer strategy e-sport...,Action/Free To Play/Strategy,free to play,https://steamcdn-a.akamaihd.net/steam/apps/570...
1,Counter-Strike: Global Offensive,730.0,"valve, hidden path entertainment",Valve,4301923.0,591081.0,0.0,"100,000,000 .. 200,000,000",27866.0,990.0,...,0.879199,https://store.steampowered.com/app/730/counter...,8.0,2012,"valve,hidden path entertainment","valve,valve",fps shooter multiplayer competitive action tea...,Action/Free To Play,free to play,https://steamcdn-a.akamaihd.net/steam/apps/730...
2,Team Fortress 2,440.0,valve,Valve,676327.0,42615.0,0.0,"50,000,000 .. 100,000,000",10154.0,1442.0,...,0.940725,https://store.steampowered.com/app/440/team_fo...,8.0,2007,valve,"valve,valve",free to play multiplayer fps shooter action cl...,Action/Free To Play,free to play,https://steamcdn-a.akamaihd.net/steam/apps/440...
3,Playerunknown'S Battlegrounds,578080.0,pubg corporation,Pubg Corporation,853709.0,732665.0,0.0,"50,000,000 .. 100,000,000",24877.0,564.0,...,0.538151,https://store.steampowered.com/app/578080/play...,5.0,2017,pubg corporation,"pubg corporation,pubg corporation",survival shooter multiplayer battle royale pvp...,Action/Adventure/Massively Multiplayer,$29.99,https://steamcdn-a.akamaihd.net/steam/apps/578...
4,Unturned,304930.0,smartly dressed games,Smartly Dressed Games,368100.0,36065.0,0.0,"20,000,000 .. 50,000,000",6736.0,4232.0,...,0.910767,https://store.steampowered.com/app/304930/untu...,8.0,2017,smartly dressed games,"smartly dressed games,smartly dressed games",free to play survival zombies multiplayer open...,Action/Adventure/Casual/Free To Play/Indie,free to play,https://steamcdn-a.akamaihd.net/steam/apps/304...


In [10]:
df['popular_tags'] = df['popular_tags'].str.replace(',', ' ')

In [12]:
cvec = CountVectorizer()
cvec_matrix = cvec.fit_transform(df['popular_tags'])

In [15]:
tfidf = TfidfVectorizer(ngram_range=(1, 3),
                        min_df=5,
                        max_df=0.75)

In [16]:
tfidf_matrix = tfidf.fit_transform(df['publisher_x'])

In [17]:
combined_matrices = hstack((tfidf_matrix, cvec_matrix))

In [18]:
recommender = pairwise_distances(combined_matrices, metric='cosine')

In [20]:
rec_df = pd.DataFrame(recommender, index = df['name'], columns = df['name'])

In [22]:
rec_df.shape

(19883, 19883)

In [24]:
rec_df["Playerunknown'S Battlegrounds"].sort_values()[1:11]

name
Realm Royale                            0.206783
Orion: Prelude                          0.235907
Tom Clancy’S Splinter Cell Blacklist    0.249806
Arma 3                                  0.264785
Rise:30 Minutes To Extinction           0.269118
Spacelords                              0.278289
Primal Carnage: Extinction              0.280853
Z1 Battle Royale                        0.280853
God Mode                                0.282781
Infected Battlegrounds                  0.283885
Name: Playerunknown'S Battlegrounds, dtype: float64

In [25]:
rec_df.head()

name,Dota 2,Counter-Strike: Global Offensive,Team Fortress 2,Playerunknown'S Battlegrounds,Unturned,Warframe,Left 4 Dead 2,Grand Theft Auto V,Terraria,Brawlhalla,...,Defenders Of The Realm Vr,Risk,Ancient Battle: Hannibal,Massive Assault: Phantom Renaissance,Hasbro'S Battleship,Royal Roads,Wars And Battles: October War,Santa'S Christmas Solitaire,Daisenryaku Perfect 4.0/大戦略パーフェクト4.0,Space Flowers
name,,,,,,,,,,,,,,,,,,,,,
Dota 2,0.000000,0.424803,0.458668,0.546461,0.683641,0.570331,0.509501,0.818166,0.547089,0.569452,...,0.886772,0.869256,0.886772,0.839872,0.853824,0.907550,0.886772,0.928389,0.855159,0.856777
Counter-Strike: Global Offensive,0.424803,0.000000,0.301747,0.330651,0.645161,0.544842,0.371245,0.621234,0.650750,0.609066,...,0.873000,0.853353,0.873000,0.820395,0.836044,0.896305,0.873000,0.919678,0.891694,0.919678
Team Fortress 2,0.458668,0.301747,0.000000,0.420454,0.544617,0.496047,0.383608,0.588694,0.641431,0.567156,...,1.000000,0.930993,1.000000,0.830969,0.845697,1.000000,1.000000,1.000000,1.000000,1.000000
Playerunknown'S Battlegrounds,0.546461,0.330651,0.420454,0.000000,0.544842,0.400000,0.412905,0.419619,0.683772,0.618268,...,0.894591,0.878284,0.894591,0.850929,0.863917,0.913934,0.894591,0.933333,0.820213,0.933333
Unturned,0.683641,0.645161,0.544617,0.544842,0.000000,0.491294,0.476037,0.498345,0.491999,0.471090,...,1.000000,0.780029,0.970110,1.000000,0.868835,0.896305,0.970110,0.903360,1.000000,0.839356


In [30]:
rec_df.to_csv('../data/genre_rating_based2.csv')

In [27]:
rec_df

name,Dota 2,Counter-Strike: Global Offensive,Team Fortress 2,Playerunknown'S Battlegrounds,Unturned,Warframe,Left 4 Dead 2,Grand Theft Auto V,Terraria,Brawlhalla,...,Defenders Of The Realm Vr,Risk,Ancient Battle: Hannibal,Massive Assault: Phantom Renaissance,Hasbro'S Battleship,Royal Roads,Wars And Battles: October War,Santa'S Christmas Solitaire,Daisenryaku Perfect 4.0/大戦略パーフェクト4.0,Space Flowers
name,,,,,,,,,,,,,,,,,,,,,
Dota 2,0.000000,0.424803,0.458668,0.546461,0.683641,0.570331,0.509501,0.818166,0.547089,0.569452,...,0.886772,0.869256,8.867723e-01,0.839872,0.853824,0.907550,0.886772,0.928389,0.855159,0.856777
Counter-Strike: Global Offensive,0.424803,0.000000,0.301747,0.330651,0.645161,0.544842,0.371245,0.621234,0.650750,0.609066,...,0.873000,0.853353,8.729999e-01,0.820395,0.836044,0.896305,0.873000,0.919678,0.891694,0.919678
Team Fortress 2,0.458668,0.301747,0.000000,0.420454,0.544617,0.496047,0.383608,0.588694,0.641431,0.567156,...,1.000000,0.930993,1.000000e+00,0.830969,0.845697,1.000000,1.000000,1.000000,1.000000,1.000000
Playerunknown'S Battlegrounds,0.546461,0.330651,0.420454,0.000000,0.544842,0.400000,0.412905,0.419619,0.683772,0.618268,...,0.894591,0.878284,8.945907e-01,0.850929,0.863917,0.913934,0.894591,0.933333,0.820213,0.933333
Unturned,0.683641,0.645161,0.544617,0.544842,0.000000,0.491294,0.476037,0.498345,0.491999,0.471090,...,1.000000,0.780029,9.701098e-01,1.000000,0.868835,0.896305,0.970110,0.903360,1.000000,0.839356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Royal Roads,0.907550,0.896305,1.000000,0.913934,0.896305,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.591752,0.528595,5.917517e-01,0.711325,0.683772,0.000000,0.591752,0.483602,0.651845,0.741801
Wars And Battles: October War,0.886772,0.873000,1.000000,0.894591,0.970110,1.000000,1.000000,0.994120,1.000000,1.000000,...,0.500000,0.711325,2.220446e-16,0.646447,0.741801,0.591752,0.000000,0.668652,0.573599,0.683772
Santa'S Christmas Solitaire,0.928389,0.919678,1.000000,0.933333,0.903360,1.000000,1.000000,0.996790,1.000000,1.000000,...,0.683772,0.452277,6.686523e-01,0.776393,0.673401,0.483602,0.668652,0.000000,0.730320,0.800000


In [3]:
new = pd.read_csv('../data/genre_rating_based2.csv')

In [7]:
import string

In [8]:
df = new.copy()

In [9]:
df

,name,Dota 2,Counter-Strike: Global Offensive,Team Fortress 2,Playerunknown'S Battlegrounds,Unturned,Warframe,Left 4 Dead 2,Grand Theft Auto V,Terraria,...,Defenders Of The Realm Vr,Risk,Ancient Battle: Hannibal,Massive Assault: Phantom Renaissance,Hasbro'S Battleship,Royal Roads,Wars And Battles: October War,Santa'S Christmas Solitaire,Daisenryaku Perfect 4.0/大戦略パーフェクト4.0,Space Flowers
0,Dota 2,0.000000,0.424803,0.458668,0.546461,0.683641,0.570331,0.509501,0.818166,0.547089,...,0.886772,0.869256,8.867723e-01,0.839872,0.853824,0.907550,0.886772,0.928389,0.855159,0.856777
1,Counter-Strike: Global Offensive,0.424803,0.000000,0.301747,0.330651,0.645161,0.544842,0.371245,0.621234,0.650750,...,0.873000,0.853353,8.729999e-01,0.820395,0.836044,0.896305,0.873000,0.919678,0.891694,0.919678
2,Team Fortress 2,0.458668,0.301747,0.000000,0.420454,0.544617,0.496047,0.383608,0.588694,0.641431,...,1.000000,0.930993,1.000000e+00,0.830969,0.845697,1.000000,1.000000,1.000000,1.000000,1.000000
3,Playerunknown'S Battlegrounds,0.546461,0.330651,0.420454,0.000000,0.544842,0.400000,0.412905,0.419619,0.683772,...,0.894591,0.878284,8.945907e-01,0.850929,0.863917,0.913934,0.894591,0.933333,0.820213,0.933333
4,Unturned,0.683641,0.645161,0.544617,0.544842,0.000000,0.491294,0.476037,0.498345,0.491999,...,1.000000,0.780029,9.701098e-01,1.000000,0.868835,0.896305,0.970110,0.903360,1.000000,0.839356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19878,Royal Roads,0.907550,0.896305,1.000000,0.913934,0.896305,1.000000,1.000000,1.000000,1.000000,...,0.591752,0.528595,5.917517e-01,0.711325,0.683772,0.000000,0.591752,0.483602,0.651845,0.741801
19879,Wars And Battles: October War,0.886772,0.873000,1.000000,0.894591,0.970110,1.000000,1.000000,0.994120,1.000000,...,0.500000,0.711325,2.220446e-16,0.646447,0.741801,0.591752,0.000000,0.668652,0.573599,0.683772
19880,Santa'S Christmas Solitaire,0.928389,0.919678,1.000000,0.933333,0.903360,1.000000,1.000000,0.996790,1.000000,...,0.683772,0.452277,6.686523e-01,0.776393,0.673401,0.483602,0.668652,0.000000,0.730320,0.800000
19881,Daisenryaku Perfect 4.0/大戦略パーフェクト4.0,0.855159,0.891694,1.000000,0.820213,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.573599,0.753817,5.735986e-01,0.698489,0.724759,0.651845,0.573599,0.730320,0.000000,0.730320


In [10]:
df.set_index('name', inplace = True)

In [11]:
df.head()

,Dota 2,Counter-Strike: Global Offensive,Team Fortress 2,Playerunknown'S Battlegrounds,Unturned,Warframe,Left 4 Dead 2,Grand Theft Auto V,Terraria,Brawlhalla,...,Defenders Of The Realm Vr,Risk,Ancient Battle: Hannibal,Massive Assault: Phantom Renaissance,Hasbro'S Battleship,Royal Roads,Wars And Battles: October War,Santa'S Christmas Solitaire,Daisenryaku Perfect 4.0/大戦略パーフェクト4.0,Space Flowers
name,,,,,,,,,,,,,,,,,,,,,
Dota 2,0.000000,0.424803,0.458668,0.546461,0.683641,0.570331,0.509501,0.818166,0.547089,0.569452,...,0.886772,0.869256,0.886772,0.839872,0.853824,0.907550,0.886772,0.928389,0.855159,0.856777
Counter-Strike: Global Offensive,0.424803,0.000000,0.301747,0.330651,0.645161,0.544842,0.371245,0.621234,0.650750,0.609066,...,0.873000,0.853353,0.873000,0.820395,0.836044,0.896305,0.873000,0.919678,0.891694,0.919678
Team Fortress 2,0.458668,0.301747,0.000000,0.420454,0.544617,0.496047,0.383608,0.588694,0.641431,0.567156,...,1.000000,0.930993,1.000000,0.830969,0.845697,1.000000,1.000000,1.000000,1.000000,1.000000
Playerunknown'S Battlegrounds,0.546461,0.330651,0.420454,0.000000,0.544842,0.400000,0.412905,0.419619,0.683772,0.618268,...,0.894591,0.878284,0.894591,0.850929,0.863917,0.913934,0.894591,0.933333,0.820213,0.933333
Unturned,0.683641,0.645161,0.544617,0.544842,0.000000,0.491294,0.476037,0.498345,0.491999,0.471090,...,1.000000,0.780029,0.970110,1.000000,0.868835,0.896305,0.970110,0.903360,1.000000,0.839356


In [12]:
df.rename(columns = string.capwords, index = string.capwords, inplace = True)

In [15]:
df.to_csv('../data/genre_rating_based2.csv')

In [14]:
df.shape

(19883, 19883)

In [13]:
df.head(10)

,Dota 2,Counter-strike: Global Offensive,Team Fortress 2,Playerunknown's Battlegrounds,Unturned,Warframe,Left 4 Dead 2,Grand Theft Auto V,Terraria,Brawlhalla,...,Defenders Of The Realm Vr,Risk,Ancient Battle: Hannibal,Massive Assault: Phantom Renaissance,Hasbro's Battleship,Royal Roads,Wars And Battles: October War,Santa's Christmas Solitaire,Daisenryaku Perfect 4.0/大戦略パーフェクト4.0,Space Flowers
name,,,,,,,,,,,,,,,,,,,,,
Dota 2,0.000000,0.424803,0.458668,0.546461,0.683641,0.570331,0.509501,0.818166,0.547089,0.569452,...,0.886772,0.869256,0.886772,0.839872,0.853824,0.907550,0.886772,0.928389,0.855159,0.856777
Counter-strike: Global Offensive,0.424803,0.000000,0.301747,0.330651,0.645161,0.544842,0.371245,0.621234,0.650750,0.609066,...,0.873000,0.853353,0.873000,0.820395,0.836044,0.896305,0.873000,0.919678,0.891694,0.919678
Team Fortress 2,0.458668,0.301747,0.000000,0.420454,0.544617,0.496047,0.383608,0.588694,0.641431,0.567156,...,1.000000,0.930993,1.000000,0.830969,0.845697,1.000000,1.000000,1.000000,1.000000,1.000000
Playerunknown's Battlegrounds,0.546461,0.330651,0.420454,0.000000,0.544842,0.400000,0.412905,0.419619,0.683772,0.618268,...,0.894591,0.878284,0.894591,0.850929,0.863917,0.913934,0.894591,0.933333,0.820213,0.933333
Unturned,0.683641,0.645161,0.544617,0.544842,0.000000,0.491294,0.476037,0.498345,0.491999,0.471090,...,1.000000,0.780029,0.970110,1.000000,0.868835,0.896305,0.970110,0.903360,1.000000,0.839356
Warframe,0.570331,0.544842,0.496047,0.400000,0.491294,0.000000,0.521626,0.467984,0.631068,0.522835,...,1.000000,0.878284,1.000000,1.000000,0.918350,1.000000,1.000000,1.000000,1.000000,0.933333
Left 4 Dead 2,0.509501,0.371245,0.383608,0.412905,0.476037,0.521626,0.000000,0.597739,0.458504,0.439717,...,1.000000,0.940451,1.000000,0.927068,0.893475,1.000000,1.000000,1.000000,1.000000,0.934767
Grand Theft Auto V,0.818166,0.621234,0.588694,0.419619,0.498345,0.467984,0.597739,0.000000,0.655876,0.750756,...,1.000000,0.933773,0.994120,1.000000,0.940765,1.000000,0.994120,0.996790,1.000000,0.927452
Terraria,0.547089,0.650750,0.641431,0.683772,0.491999,0.631068,0.458504,0.655876,0.000000,0.456786,...,1.000000,0.927831,1.000000,1.000000,0.870901,1.000000,1.000000,1.000000,1.000000,0.841886


In [28]:
df["Playerunknown's Battlegrounds"].sort_values()[1:13]

name
Realm Royale                            0.206783
Orion: Prelude                          0.235907
Tom Clancy’s Splinter Cell Blacklist    0.249806
Arma 3                                  0.264785
Rise:30 Minutes To Extinction           0.269118
Spacelords                              0.278289
Primal Carnage: Extinction              0.280853
Z1 Battle Royale                        0.280853
God Mode                                0.282781
Infected Battlegrounds                  0.283885
Darwin Project                          0.286932
Primal Carnage                          0.286932
Name: Playerunknown's Battlegrounds, dtype: float64

In [16]:
ddf = pd.read_csv('../data/final_df2.csv')

In [18]:
nnew = ddf.copy()

In [19]:
ddf.set_index('name', inplace = True)

In [20]:
ddf.head()

,Unnamed: 0,Unnamed: 0.1,appid,developer_x,publisher_x,positive,negative,userscore,owners,average_forever,...,pos/neg rate,url,all_reviews,release_date,developer_y,publisher_y,popular_tags,genre,original_price,images
name,,,,,,,,,,,,,,,,,,,,,
Dota 2,0,0,570.0,valve,Valve,1173499.0,214974.0,0.0,"100,000,000 .. 200,000,000",31355.0,...,0.845172,https://store.steampowered.com/app/570/dota_2/,8.0,2013,valve,"valve,valve",free to play moba multiplayer strategy e-sport...,Action/Free To Play/Strategy,free to play,https://steamcdn-a.akamaihd.net/steam/apps/570...
Counter-Strike: Global Offensive,1,1,730.0,"valve, hidden path entertainment",Valve,4301923.0,591081.0,0.0,"100,000,000 .. 200,000,000",27866.0,...,0.879199,https://store.steampowered.com/app/730/counter...,8.0,2012,"valve,hidden path entertainment","valve,valve",fps shooter multiplayer competitive action tea...,Action/Free To Play,free to play,https://steamcdn-a.akamaihd.net/steam/apps/730...
Team Fortress 2,2,2,440.0,valve,Valve,676327.0,42615.0,0.0,"50,000,000 .. 100,000,000",10154.0,...,0.940725,https://store.steampowered.com/app/440/team_fo...,8.0,2007,valve,"valve,valve",free to play multiplayer fps shooter action cl...,Action/Free To Play,free to play,https://steamcdn-a.akamaihd.net/steam/apps/440...
Playerunknown'S Battlegrounds,3,3,578080.0,pubg corporation,Pubg Corporation,853709.0,732665.0,0.0,"50,000,000 .. 100,000,000",24877.0,...,0.538151,https://store.steampowered.com/app/578080/play...,5.0,2017,pubg corporation,"pubg corporation,pubg corporation",survival shooter multiplayer battle royale pvp...,Action/Adventure/Massively Multiplayer,$29.99,https://steamcdn-a.akamaihd.net/steam/apps/578...
Unturned,4,4,304930.0,smartly dressed games,Smartly Dressed Games,368100.0,36065.0,0.0,"20,000,000 .. 50,000,000",6736.0,...,0.910767,https://store.steampowered.com/app/304930/untu...,8.0,2017,smartly dressed games,"smartly dressed games,smartly dressed games",free to play survival zombies multiplayer open...,Action/Adventure/Casual/Free To Play/Indie,free to play,https://steamcdn-a.akamaihd.net/steam/apps/304...


In [21]:
ddf.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'], inplace = True)

In [22]:
ddf.head()

,appid,developer_x,publisher_x,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,...,pos/neg rate,url,all_reviews,release_date,developer_y,publisher_y,popular_tags,genre,original_price,images
name,,,,,,,,,,,,,,,,,,,,,
Dota 2,570.0,valve,Valve,1173499.0,214974.0,0.0,"100,000,000 .. 200,000,000",31355.0,1943.0,973.0,...,0.845172,https://store.steampowered.com/app/570/dota_2/,8.0,2013,valve,"valve,valve",free to play moba multiplayer strategy e-sport...,Action/Free To Play/Strategy,free to play,https://steamcdn-a.akamaihd.net/steam/apps/570...
Counter-Strike: Global Offensive,730.0,"valve, hidden path entertainment",Valve,4301923.0,591081.0,0.0,"100,000,000 .. 200,000,000",27866.0,990.0,7640.0,...,0.879199,https://store.steampowered.com/app/730/counter...,8.0,2012,"valve,hidden path entertainment","valve,valve",fps shooter multiplayer competitive action tea...,Action/Free To Play,free to play,https://steamcdn-a.akamaihd.net/steam/apps/730...
Team Fortress 2,440.0,valve,Valve,676327.0,42615.0,0.0,"50,000,000 .. 100,000,000",10154.0,1442.0,403.0,...,0.940725,https://store.steampowered.com/app/440/team_fo...,8.0,2007,valve,"valve,valve",free to play multiplayer fps shooter action cl...,Action/Free To Play,free to play,https://steamcdn-a.akamaihd.net/steam/apps/440...
Playerunknown'S Battlegrounds,578080.0,pubg corporation,Pubg Corporation,853709.0,732665.0,0.0,"50,000,000 .. 100,000,000",24877.0,564.0,10375.0,...,0.538151,https://store.steampowered.com/app/578080/play...,5.0,2017,pubg corporation,"pubg corporation,pubg corporation",survival shooter multiplayer battle royale pvp...,Action/Adventure/Massively Multiplayer,$29.99,https://steamcdn-a.akamaihd.net/steam/apps/578...
Unturned,304930.0,smartly dressed games,Smartly Dressed Games,368100.0,36065.0,0.0,"20,000,000 .. 50,000,000",6736.0,4232.0,353.0,...,0.910767,https://store.steampowered.com/app/304930/untu...,8.0,2017,smartly dressed games,"smartly dressed games,smartly dressed games",free to play survival zombies multiplayer open...,Action/Adventure/Casual/Free To Play/Indie,free to play,https://steamcdn-a.akamaihd.net/steam/apps/304...


In [23]:
ddf.rename(index = string.capwords, inplace = True)

In [24]:
ddf.index

Index(['Dota 2', 'Counter-strike: Global Offensive', 'Team Fortress 2',
       'Playerunknown's Battlegrounds', 'Unturned', 'Warframe',
       'Left 4 Dead 2', 'Grand Theft Auto V', 'Terraria', 'Brawlhalla',
       ...
       'Defenders Of The Realm Vr', 'Risk', 'Ancient Battle: Hannibal',
       'Massive Assault: Phantom Renaissance', 'Hasbro's Battleship',
       'Royal Roads', 'Wars And Battles: October War',
       'Santa's Christmas Solitaire', 'Daisenryaku Perfect 4.0/大戦略パーフェクト4.0',
       'Space Flowers'],
      dtype='object', name='name', length=19883)

In [25]:
ddf.to_csv('../data/final_df2.csv')